## 1. Setting Kaggle's Datasets API

This is our first notebook. So first, let's get a Kaggle API key, and create  easy access to download Kaggle's datasets.
More details on Kaggle's API can be found in the following [link](https://github.com/Kaggle/kaggle-api#api-credentials). Moreover, a quick explanation on how to get an API key and use it can be found in the following [link](https://gist.github.com/jayspeidell/d10b84b8d3da52df723beacc5b15cb27)

In [ ]:
!mkdir /root/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
import json
import os

# Installing the Kaggle package
!pip install kaggle 


# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Let's check if we can use Kaggle API and download the [US Baby Names  dataset](https://www.kaggle.com/kaggle/us-baby-names)

In [ ]:
# searching for the dataset
!kaggle datasets list -s baby

ref                                           title                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------  --------------------------------  -----  -------------------  -------------  ---------  ---------------  
kaggle/us-baby-names                          US Baby Names                     173MB  2017-11-21 22:18:15          30674        323  0.5882353        
thedevastator/us-baby-names-by-year-of-birth  Baby Names by Year                  9MB  2022-09-20 06:26:33            752         35  1.0              
ironicninja/baby-names                        Baby Names                          5MB  2021-02-09 16:21:40            455         12  0.88235295       
haezer/french-baby-names                      French Baby Names                  18MB  2020-04-07 16:32:38            429         15  0.88235295       
robikscube/us-baby-name-popularity            US Baby Name Popularity            52MB  2

In [ ]:
# Creating a dataset directory
!mkdir ./datasets
!mkdir ./datasets/us-baby-name

# download the dataset from Kaggle and unzip it
!kaggle datasets download kaggle/us-baby-names -f database.sqlite -p ./datasets/us-baby-name/database.sqlite
!chdir ./datasets/us-baby-name/database.sqlite/
!unzip ./datasets/us-baby-name/database.sqlite/*.zip  -d ./datasets/us-baby-name/database.sqlite

 88% 116M/131M [00:00<00:00, 214MB/s] 
100% 131M/131M [00:00<00:00, 226MB/s]
/bin/bash: chdir: command not found
Archive:  ./datasets/us-baby-name/database.sqlite/database.sqlite.zip
  inflating: ./datasets/us-baby-name/database.sqlite/database.sqlite  


## 2. Working with SQLite

In the first part, we downloaded an SQLite database into Colab. Let's connect to the dataset using the [sqlite3 package](https://docs.python.org/2/library/sqlite3.html): First, let's import the required packages, and answer several questions about the data.

In [ ]:
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline

DB_PATH = './datasets/us-baby-name/database.sqlite'

**Question 1: How many names in the dataset?**

In [ ]:
conn = sqlite3.connect(f'{DB_PATH}/database.sqlite') # connecting to the database
c = conn.cursor() # creating a cursor object
print("The number of national names in the dataset: %s" % c.execute("SELECT count(*) FROM NationalNames" ).fetchone()) # execute a query & fetch the results
print("The number of state names in the dataset: %s" % c.execute("SELECT count(*) FROM StateNames" ).fetchone()) 
c.close() # close the cursor

The number of national names in the dataset: 1825433
The number of state names in the dataset: 5647426


**Question 2: How many distinct names in the Washington State? and in New York State?**

In [ ]:
c = conn.cursor()
print(c.execute("SELECT COUNT(DISTINCT(Name)) as count FROM StateNames WHERE State='NY' " ).fetchone()[0])
c.close()

13933


**Question 3: What are the most rare names?**

In [ ]:
c = conn.cursor()  
rows = c.execute("""SELECT Name, Sum(Count) as TotalNumberOfBabies
             FROM NationalNames
             Group By Name
             Order by Sum(Count)""" ) 
i = 1 
for r in rows:
  if i > 20:
    break
  print(f"{i}. {r[0]}, {r[1]}")
  #print({0}, {1}, {2}.format(i, r[0], r[1]))
  i += 1
c.close() # close the cursor

1. Aabid, 5
2. Aadhyan, 5
3. Aadian, 5
4. Aadrian, 5
5. Aadrit, 5
6. Aafreen, 5
7. Aagot, 5
8. Aahron, 5
9. Aaiyana, 5
10. Aaja, 5
11. Aakanksha, 5
12. Aakira, 5
13. Aakiyah, 5
14. Aaleeya, 5
15. Aalias, 5
16. Aalicia, 5
17. Aaliyaa, 5
18. Aallyah, 5
19. Aalon, 5
20. Aalyia, 5


Now let's print the names in descending order:

**Question 4: What are the most common names?**

In [ ]:
c = conn.cursor()  
rows = c.execute("""SELECT Name, Sum(Count) as TotalNumberOfBabies
             FROM NationalNames
             Group By Name
             Order by Sum(Count) DESC""" )

 
i = 1 
for r in rows:
  if i > 20:
    break
  print(f"{i}. {r[0]}, {r[1]}")
  #print({0}, {1}, {2}.format(i, r[0], r[1]))
  i += 1
c.close() # close the cursor

1. James, 5129096
2. John, 5106590
3. Robert, 4816785
4. Michael, 4330805
5. Mary, 4130441
6. William, 4071368
7. David, 3590557
8. Joseph, 2580687
9. Richard, 2564867
10. Charles, 2376700
11. Thomas, 2291517
12. Christopher, 2004177
13. Daniel, 1876880
14. Elizabeth, 1606282
15. Patricia, 1575529
16. Matthew, 1558671
17. Jennifer, 1467573
18. George, 1464430
19. Linda, 1454599
20. Barbara, 1437083


**Question 5:** Write a code that queries the babies DB you prepared and retrieves the number of male babies named George in 1992.

In [ ]:
c = conn.cursor()
print(c.execute("SELECT COUNT FROM NationalNames WHERE Name=\"George\" and Year=1992 and Gender=\"M\"").fetchone()[0])
c.close()

3892


***Question 6:*** Write a function that returns how many babies were born in a given state in a given year.
Use it to calculate the number of babies born in TX in 1986.

In [ ]:
def get_count(country,year):
  c = conn.cursor()
  return_value= c.execute("SELECT SUM(Count) from StateNames WHERE State=? and Year=?",(country,year)).fetchone()[0]
  c.close()
  return return_value

print(get_count("TX",'1986'))

277779


**Question 7:** Write a function that returns how many male babies named Robert where born in a given state in a given year. 
Use it to find the state in which the highest number of babies Robert were born in 2010 (10pt)

In [ ]:

def get_count(name,year):
  c = conn.cursor()
  return_value =  c.execute("SELECT max(count),State FROM (SELECT sum(Count) as count,State FROM StateNames WHERE (Name=? AND Year=?) Group By State)",(name,year)).fetchone()[1]
  return return_value

print(get_count("Robert",2010))


CA


## 3. Working with Pony ORM

In [ ]:
!pip install pony

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets list -s pokemon

ref                                                 title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
abcsds/pokemon                                      Pokemon with stats                            15KB  2016-08-29 06:01:43          93876       2276  0.88235295       
vishalsubbiah/pokemon-images-and-types              Pokemon Image Dataset                          2MB  2018-12-17 01:01:39          19849        431  0.9411765        
rounakbanik/pokemon                                 The Complete Pokemon Dataset                  47KB  2017-09-29 19:49:32          47442        985  0.8235294        
kvpratama/pokemon-images-dataset                    Pokemon Images Dataset                        71MB  2020-08-10 02:05:00          11573        254  0.87

In [ ]:
!kaggle datasets download abcsds/pokemon -p ./datasets/
!unzip ./datasets/pokemon.zip -d ./datasets/pokemon/

  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 906kB/s]
Archive:  ./datasets/pokemon.zip
  inflating: ./datasets/pokemon/Pokemon.csv  


In [ ]:
!ls ./datasets/pokemon

Pokemon.csv


### 3.1 Creating DB using Objects

Now let's use the Pokemon dataset and Pony to create a database. The first step is to create classes that will map to tables in the generate database:

In [ ]:
from pony.orm import *
# Creating a new database
db = Database()
db.bind(provider='sqlite', filename='/content/datasets/pokemon/pokemon.pony.db', create_db=True) # notice we can easily use other type of databases instead of SQLite

# Each class will be map to a table
class Trainer(db.Entity):
    name = Required(str)
    age = Required(int)
    pokemons_list = Set('Pokemon') # Connected to the Pokemon class
    
class Pokemon(db.Entity):
    name = Required(str) # cannot be None
    age  = Optional(int)
    attack_power = Required(int)
    defense_power = Required(int)
    trainer = Optional(Trainer) # Connected to the Trainer class

In [ ]:
show(Trainer)

class Trainer(Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str)
    age = Required(int)
    pokemons_list = Set(Pokemon)


In [ ]:
set_sql_debug(True) # helps to see what SQL commands are running
db.generate_mapping(create_tables=True) # create tables

We can see that Pony create the Pokemon & the Trainer tables in the database. Next, we will use the Pandas package to insert data into these tables:

In [ ]:
import pandas as pd 
df = pd.read_csv('./datasets/pokemon/Pokemon.csv')
df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Let's iterate on the dataframe rows and create a Pokemon object from each row:

In [ ]:
import random

for idx, row in df.iterrows():
    Pokemon(name=row['Name'],
            age=random.randint(0,500),
            attack_power=row['Sp. Atk'], 
            defense_power=row['Sp. Def'])

We have created 800 objects, one for each row. However, none of them was committed to the DB yet. Let's commit!

In [ ]:
commit()

Now let's add Ash as a trainer object, and add two pokemons trained by Ash:

In [ ]:
ash_trainer = Trainer(name='Ash Ketchum', age=10)
pikachu = Pokemon(name='pikachu',age=6,attack_power=50, defense_power=50, trainer=ash_trainer)
charmander =  Pokemon(name='charmander',age=2,attack_power=60, defense_power=50,trainer=ash_trainer)
commit()

We can now easily retrieve data using objects:

In [ ]:
pikachu.trainer.age

10

In [ ]:
list(pikachu.trainer.pokemons_list)

[Pokemon[801], Pokemon[802]]

In [ ]:
list(pikachu.trainer.pokemons_list)[1].name

'charmander'

We can also simply update the database using the created objects:

In [ ]:
pikachu.age

6

In [ ]:
pikachu.age = 110
commit()

In [ ]:
pikachu.age

110

### 3.2 Creating  Objects from DB:

In [ ]:
!pip install pony

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Using Pony, we can also connect to existing database, and easily search it without even knowing SQL. First, let's connect between Pony to an existing database:

In [ ]:
!pip install pony

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pony.orm import *
db = Database()
db.bind(provider='sqlite', filename='/content/datasets/pokemon/pokemon.pony.db') # notice we don't use create_db=True here

class Trainer(db.Entity):
    name = Required(str)
    age = Required(int)
    pokemons_list = Set('Pokemon')
    
class Pokemon(db.Entity):
    name = Required(str) # cannot be None
    age  = Optional(int)
    attack_power = Required(int)
    defense_power = Required(int)
    trainer = Optional(Trainer)
db.generate_mapping()
set_sql_debug(True)

**Question 1:** Print the names of Pokemons which their attack is higher than 100 and their defence is higher than attack.  

In [ ]:
[i.name for i in select(p for p in Pokemon if p.attack_power >100 and p.defense_power > p.attack_power )]

['Ho-oh',
 'MetagrossMega Metagross',
 'Latias',
 'LatiasMega Latias',
 'RotomHeat Rotom',
 'RotomWash Rotom',
 'RotomFrost Rotom',
 'RotomFan Rotom',
 'RotomMow Rotom',
 'Florges',
 'Sylveon',
 'Goodra']

We can observe that behind the scenes Pony translate the query into a SQL query and returned a list with the names of the matched pokemons. 


**Question 2:** retrieve all the pokemon names that their attack is between 50 to 100.

In [ ]:
[i.name for i in select(p for p in Pokemon if p.attack_power >50 and p.attack_power <100 )]

['Bulbasaur',
 'Ivysaur',
 'Charmander',
 'Charmeleon',
 'Wartortle',
 'Blastoise',
 'Butterfree',
 'Pidgeot',
 'Fearow',
 'Arbok',
 'Raichu',
 'Nidorina',
 'Nidoqueen',
 'Nidorino',
 'Nidoking',
 'Clefairy',
 'Clefable',
 'Ninetales',
 'Wigglytuff',
 'Golbat',
 'Oddish',
 'Gloom',
 'Parasect',
 'Venomoth',
 'Persian',
 'Psyduck',
 'Golduck',
 'Primeape',
 'Growlithe',
 'Poliwrath',
 'Machamp',
 'Bellsprout',
 'Weepinbell',
 'Tentacruel',
 'Golem',
 'Ponyta',
 'Rapidash',
 'Magnemite',
 "Farfetch'd",
 'Dodrio',
 'Dewgong',
 'Muk',
 'Cloyster',
 'Hypno',
 'Voltorb',
 'Electrode',
 'Exeggcute',
 'Lickitung',
 'Koffing',
 'Weezing',
 'KangaskhanMega Kangaskhan',
 'Horsea',
 'Seadra',
 'Seaking',
 'Staryu',
 'Scyther',
 'Electabuzz',
 'Pinsir',
 'PinsirMega Pinsir',
 'Gyarados',
 'GyaradosMega Gyarados',
 'Lapras',
 'Flareon',
 'Porygon',
 'Omanyte',
 'Kabuto',
 'Kabutops',
 'Aerodactyl',
 'AerodactylMega Aerodactyl',
 'Snorlax',
 'Articuno',
 'Dragonair',
 'Bayleef',
 'Meganium',
 'Cyndaq

**Question 3**: find the pokemon with the highest attack power:

In [ ]:
needed_power=max([i.attack_power for i in select(p for p in Pokemon)])
[i.name for i in select(p for p in Pokemon if p.attack_power ==needed_power )]

['MewtwoMega Mewtwo Y']

Using the [Kickstarter Projects Dataset](https://www.kaggle.com/kemical/kickstarter-projects#ks-projects-201801.csv), answer the following questions:

**Question 1**: Load the dataset to SQLite DB using PonyORM.

In [ ]:
!wget https://github.com/maniknarang/kickstarter-data-analysis/raw/master/ks-projects-201801.csv

--2022-11-07 13:30:00--  https://github.com/maniknarang/kickstarter-data-analysis/raw/master/ks-projects-201801.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/maniknarang/kickstarter-data-analysis/master/ks-projects-201801.csv [following]
--2022-11-07 13:30:00--  https://raw.githubusercontent.com/maniknarang/kickstarter-data-analysis/master/ks-projects-201801.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58030359 (55M) [text/plain]
Saving to: ‘ks-projects-201801.csv.1’

ks-projects-201801. 100%[===================>]  55.34M   149MB/s    in 0.4s    

2022-11-07 13:30:01 (149 MB/s) - ‘k

In [ ]:
import pandas as pd 

df = pd.read_csv("ks-projects-201801.csv",encoding="utf-8")
df

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378656,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17,50000.0,2014-09-17 02:35:30,25.0,canceled,1,US,25.0,25.0,50000.00
378657,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19,1500.0,2011-06-22 03:35:14,155.0,failed,5,US,155.0,155.0,1500.00
378658,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16,15000.0,2010-07-01 19:40:30,20.0,failed,1,US,20.0,20.0,15000.00
378659,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13,15000.0,2016-01-13 18:13:53,200.0,failed,6,US,200.0,200.0,15000.00


**Question 1:** Use Pony for creating a new database.

The database should include a Project table with a name, category and country as strings, whreas pledged and goal as floats.
Fill the table using the CSV file.

In [ ]:
from pyparsing.helpers import string
from pony.orm import *
import random
# Creating a new database
db = Database()
db.bind(provider='sqlite', filename='/content/datasets/project.pony.db', create_db=True) # notice we can easily use other type of databases instead of SQLite

class Project(db.Entity):
    name = Required(str)
    category = Required(str)
    country = Required(str)
    pledged = Required(float)
    goal = Required(float)


set_sql_debug(True)
db.generate_mapping(create_tables=True)

for idx, row in df.iterrows():
    Project(name=str(row['name']),category=row['category'],
            country=row['country'], 
            pledged=row['pledged'],
            goal=row['goal'])

commit()


**Question 2**: Write a Pony code which retrieve the project's names of those which their contry is "US".

In [ ]:
[i.name for i in select(p for p in Project if p.country =="US")]

['Greeting From Earth: ZGAC Arts Capsule For ET',
 'Where is Hank?',
 'ToshiCapital Rekordz Needs Help to Complete Album',
 'Community Film Project: The Art of Neighborhood Filmmaking',
 'Monarch Espresso Bar',
 'Support Solar Roasted Coffee & Green Energy!  SolarCoffee.co',
 'Chaser Strips. Our Strips make Shots their B*tch!',
 'SPIN - Premium Retractable In-Ear Headphones with Mic',
 'STUDIO IN THE SKY - A Documentary Feature Film (Canceled)',
 'Lisa Lim New CD!',
 'The Cottage Market',
 'G-Spot Place for Gamers to connect with eachother & go pro!',
 'Survival Rings',
 'The Beard',
 'Notes From London: Above & Below',
 "Mike Corey's Darkness & Light Album",
 'Boco Tea',
 'CMUK. Shoes: Take on Life Feet First.',
 'Alice in Wonderland in G Minor',
 'The Book Zoo - A Mini-Comic',
 'Matt Cavenaugh & Jenny Powers make their 1st album!',
 'My Moon - Animated Short Film',
 'Ledr workbook: one tough journal!',
 'BB130A',
 'Chris Eger Band - New Nashville Record!',
 'Squatch Watchers',
 'Arro